# Single Atom Calculation

In [ ]:
# Uncomment the next line if you have ipympl installed and want interactive plots
# %matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

import pairinteraction.$PI_DTYPE as pi
from pairinteraction.visualization.colormaps import alphamagma

## State of Interest

In [ ]:
ket = pi.KetAtom($SPECIES_0$QUANTUM_NUMBERS_0)
ket_energy = ket.get_energy("GHz")
print(f"State of interest: {ket}")

## Basis

In [ ]:
basis = pi.BasisAtom(ket.species$QUANTUM_NUMBERS_RESTRICTIONS_0)
print(str(basis))
print(f" ⇒ Basis consists of {basis.number_of_kets} kets")

## System

In [ ]:
steps = $STEPS
efield_x = np.linspace($EX_MIN, $EX_MAX, steps)
efield_y = np.linspace($EY_MIN, $EY_MAX, steps)
efield_z = np.linspace($EZ_MIN, $EZ_MAX, steps)
bfield_x = np.linspace($BX_MIN, $BX_MAX, steps)
bfield_y = np.linspace($BY_MIN, $BY_MAX, steps)
bfield_z = np.linspace($BZ_MIN, $BZ_MAX, steps)

In [ ]:
systems = []
for i in range(steps):
    system = pi.SystemAtom(basis)
    system.set_electric_field([efield_x[i], efield_y[i], efield_z[i]], unit="V/cm")
    system.set_magnetic_field([bfield_x[i], bfield_y[i], bfield_z[i]], unit="G")
    system.set_diamagnetism_enabled($DIAMAGNETISM_ENABLED)
    systems.append(system)

pi.diagonalize(systems$DIAGONALIZE_KWARGS$DIAGONALIZE_ENERGY_RANGE_KWARGS)

In [ ]:
energies_list = [system.get_eigenenergies(unit="GHz") - ket.get_energy("GHz") for system in systems]
overlaps_list = [system.get_eigenbasis().get_overlaps(ket) for system in systems]

## Plot

In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel($X_LABEL)
ax.set_ylabel("Energy [GHz]")

try:
    ax.plot($X_VARIABLE_NAME, np.array(energies_list), c="0.9", lw=0.25, zorder=-10)
except ValueError:  # inhomogeneous shape -> no simple line plot possible
    for x, es in zip($X_VARIABLE_NAME, energies_list):
        ax.plot([x] * len(es), es, c="0.9", ls="None", marker=".", zorder=-10)

x_repeated = np.hstack([val * np.ones_like(es) for val, es in zip($X_VARIABLE_NAME, energies_list)])
energies_flattened = np.hstack(energies_list)
overlaps_flattened = np.hstack(overlaps_list)
sorter = np.argsort(overlaps_flattened)

scat = ax.scatter(
    x_repeated[sorter],
    energies_flattened[sorter],
    c=overlaps_flattened[sorter],
    s=15,
    vmin=0,
    vmax=1,
    cmap=alphamagma
)

fig.colorbar(scat, ax=ax, label="Overlap with state of interest")

plt.show()